In [1]:
##### import sys
import re
import gc
import sys
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_log_error

sys.path.append("../")
from general.preprocess import data_preparation
from general.clf_wrappers import LgbWrapper
from features.f0 import features_set_f0

%load_ext autoreload
%autoreload 2

In [2]:
full_data, ntrain, ntest = data_preparation()

air_store dataframe shape: (829, 57)


In [4]:
trn = full_data[:ntrain]
tst = full_data[ntrain:]
print('train data size:', trn.shape, 'test data size:', tst.shape)

train data size: (252108, 123) test data size: (32019, 123)


In [12]:
TARGET = 'visitors'
FEATURES = features_set_f0()
SEED = 177
lgb_params = {}
lgb_params['n_estimators'] = 50
lgb_params['learning_rate'] = 0.1 # shrinkage_rate
lgb_params['metric'] = 'l1'          # or 'mae'
lgb_params['sub_feature'] = 0.34
lgb_params['bagging_fraction'] = 0.85 # sub_row
lgb_params['bagging_freq'] = 40
lgb_params['num_leaves'] = 512        # num_leaf
lgb_params['min_data'] = 500         # min_data_in_leaf
lgb_params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
lgb_params['verbose'] = 0
lgb_params['feature_fraction_seed'] = 2
lgb_params['bagging_seed'] = 3

In [13]:
from sklearn.metrics import mean_squared_log_error


def score_valid(y_true, y_valid):
    score = np.sqrt(mean_squared_log_error(y_true, y_valid))
    return score

def get_store_ids():
    df = pd.read_csv('../data/air_store_info.csv')
    return df['air_store_id']

In [25]:
def cross_validate(full_data, clf, seed, ntrain, ntest, features, target, nfolds=5):
    # prepare datasets used
    trn = full_data[:ntrain]
    tst = full_data[ntrain:]
    
    x_tst = tst[features].values

    # define
    oof_trn = np.zeros((ntrain,))
    oof_tst = np.zeros((ntest,))
    oof_tst_fld = np.empty((ntest, nfolds))
    oof_score = [None] * 2

    folds = KFold(n_splits=nfolds, shuffle=True, random_state=seed).split(trn.id)

    for i, ids in enumerate(folds):
        x_trn = trn.loc[ids[0], ][features].values
        y_trn = trn.loc[ids[0], ][target].values
        
        x_vld = trn.loc[ids[1], ][features].values
        y_vld = trn.loc[ids[1], ][features].values
        
        clf.train(x_trn, y_trn, x_vld, y_vld)
        
        oof_trn[ids[1]] = clf.predict(x_vld)
        oof_tst_fld[:, i] = clf.predict(x_tst)

        del x_trn, y_trn, x_vld, y_vld
        gc.collect()

    oof_tst[:] = oof_tst_fld.mean(axis=1)
    oof_score[0] = score_valid(trn[target].values, oof_trn)
    oof_score[1] = 0.1234

    sub = tst[['id', target]]
    trn = trn[['id', target]]
    
    sub[target] = oof_tst.reshape(-1, 1)
    trn[target] = oof_trn.reshape(-1, 1)
    
    return sub, trn, None, oof_score

In [26]:
lgb_clf= LgbWrapper(seed=SEED, params=lgb_params)
results = cross_validate(
    full_data=full_data,
    clf=lgb_clf,
    seed=SEED,
    ntrain=ntrain,
    ntest=ntest,
    features=FEATURES,
    target=TARGET,
    nfolds=5,
)
print(1)

1


G:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
results[3]

[0.48625300908900526, 0.1234]